In [ ]:
'''
Defining all the global variables in this cell
'''
IMG_HEIGHT = 28
IMG_WIDTH = 28
IMG_DEPTH = 1

G_WIN_SIZE= 8
G_DIM = G_WIN_SIZE*G_WIN_SIZE*IMG_DEPTH

LOC_DIM = 2 # the number of dimensions for the locations are just x and y so 2
GLIMPSE_FC1 = 128
GLIMPSE_FC2 = 256

LSTM_HIDDEN = 256

NUM_GLIMPSES = 4



In [ ]:
class Model:
    def __init__(self, inputs, b_size):
        self.inputs= inputs
        self.batch_size = b_size
        
    def __call__(self, inputs, b_size)
        # Normal instead of uniform 
        initial_locs = tf.random_uniform([batch_size, LOC_DIM], minval=-1, maxval=1)
        input_lstm = self.glimpse_network(self.inputs, initial_locs)

        lstm_cell  = tf.nn.rnn_cell.LSTMCell(LSTM_HIDDEN, state_is_tuple=True)
        init_state = lstm_cell.zero_state(batch_size, tf.float32)

        inputs = np.zeros((NUM_GLIMPSES, batch_size, GLIMPSE_FC2))
        inputs[0] = input_lstm

        outputs, state = tf.contrib.legacy_seq2seq.rnn_decoder(inputs, init_state, 
                                                               lstm_cell, loop_function=next_location)

        return outputs
    

    def next_location(self, prev_inputs, i):
        # Change this function to have all the variables defined here so that stop gradients can be applied.
        locs = self.fc_layer(prev_inputs, GLIMPSE_FC2, LOC_DIM, 'next_loc', None)
        locs = tf.clip_by_value(next_loc, -1., 1.)
        locs = tf.stop_gradient(next_loc)

        next_inputs = self.glimpse_network(self.inputs, locs)

        return next_inputs


    def glimpse_network(self, input_img, locations):

        loc_out1 = self.fc_layer(locations, LOC_DIM, GLIMPSE_FC1, 'lc1', tf.nn.relu)
        loc_out2 = self.fc_layer(loc_out1, GLIMPSE_FC1, GLIMPSE_FC2, 'lc2', tf.nn.relu)

        glimpses = tf.image.extract_glimpse(input_img, [G_WIN_SIZE,G_WIN_SIZE], 
                                            locations, centered=True, normalized=True)

        glimpses = tf.reshape(glimpses, [-1, G_WIN_SIZE*G_WIN_SIZE*IMG_DEPTH])

        g_out1 = self.fc_layer(glimpses, G_DIM, GLIMPSE_FC1, 'g1', tf.nn.relu)
        g_out2 = self.fc_layer(g_out1, GLIMPSE_FC1, GLIMPSE_FC2, 'g2', tf.nn.relu)

        return tf.nn.relu(loc_out2 + g_out2)


    def fc_layer(self, image, in_size, out_size, name, activation):
        with tf.variable_scope(name):
            weights = tf.get_variable("weights", [in_size, out_size], initializer=tf.contrib.layers.xavier_initializer())
            biases = tf.get_variable("biases", [out_size], initializer=tf.contrib.layers.xavier_initializer())
            y = tf.add(tf.matmul(image, weights), biases)
            tf.summary.histogram('weights_hist', weights)
            tf.summary.histogram('biases_hist', biases)

            if activation is not None:
                y = activation(y)

            return y
    
        

In [ ]:
'''
Train function
Processes inputs in mini-batchs 
Builds the model and trains the parameters for predetermined number of times 
'''

import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data
import os 
from tensorflow.python.framework import ops
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID" 
os.environ["CUDA_VISIBLE_DEVICES"] = "0"


def train(batch_size, epochs, log, output):
    # Read arguments
    ops.reset_default_graph()
    mnist = input_data.read_data_sets('MNIST_data', one_hot=True)
    # placeholders for the input and labels
    X = tf.placeholder(tf.float32, [None, INPUT_SIZE], name='X')
    y = tf.placeholder(tf.float32, [None, 10], name='labels')

    y_hat = model(X, batch_size)

    '''
    Calculate loss and then define the Optimizer and related hyper-parameters
    '''
    saver = tf.train.Saver()

    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    config.gpu_options.per_process_gpu_memory_fraction = 0.33
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())

        merged_summary = tf.summary.merge_all()
        writer = tf.summary.FileWriter(log, sess.graph)

        coord = tf.train.Coordinator()
        threads = tf.train.start_queue_runners(sess=sess, coord=coord)
        # training the model for a predetermined number of epochs
        for i in range(epochs):
            batch = mnist.train.next_batch(batch_size)
            acc = sess.run(y_hat, feed_dict={X: batch[0], y: batch[1]})
            s = sess.run(merged_summary, feed_dict={X: batch[0], y: batch[1]})
            writer.add_summary(s, i)

            if (i+1) % 275 == 0:
                print('Step {}: {:.2f}'.format(i+1, acc))

            if (((i+1) % 275 == 0) and (acc > 0.99)):
                # Please change the directory here to save the model in a different location
                params = saver.save(sess, '/N/u/ramyarao/dl_hw2/exp_model/{}_{}.ckpt'.format(output, i+1))
                print('Model saved: {}'.format(params))

        coord.request_stop()
        coord.join(threads)
    return
        

if __name__ == '__main__':
    train(200, 275*10000, 'logs', 'model')


